In [2]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import sqlite3

import pyspark
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.classification import (DecisionTreeClassifier, RandomForestClassifier, GBTClassifier, LogisticRegression)
from pyspark.ml.evaluation import (BinaryClassificationEvaluator, MulticlassClassificationEvaluator)
from pyspark.mllib.evaluation import (BinaryClassificationMetrics, MulticlassMetrics)
from pyspark.ml import Pipeline
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql.functions import lit
import pyspark.sql.functions as F

import warnings
warnings.filterwarnings('ignore')

In [3]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

21/12/16 17:54:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/16 17:54:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/12/16 17:54:24 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
21/12/16 17:54:24 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [4]:
#Transaction data is stored in sqlite db

connection = sqlite3.connect('data/credit_card_tran_data.db')

df1 = pd.read_sql_query("SELECT * FROM part1_data", connection)
df2 = pd.read_sql_query("SELECT * FROM part2_data", connection)

In [5]:
df1.shape

(555719, 23)

In [6]:
df2.shape

(1296675, 23)

In [7]:
data_df = pd.concat([df1, df2])

In [8]:
data_df.shape

(1852394, 23)

In [9]:
data_df.head(10)

,field1,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2020-06-21 12:14:25,2.291164e+15,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,1,2020-06-21 12:14:33,3.573030e+15,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2,2020-06-21 12:14:53,3.598215e+15,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,...,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,3,2020-06-21 12:15:15,3.591920e+15,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,...,28.5697,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,4,2020-06-21 12:15:17,3.526826e+15,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,...,44.2529,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0
5,5,2020-06-21 12:15:37,3.040768e+13,fraud_Daugherty LLC,kids_pets,19.55,Danielle,Evans,F,76752 David Lodge Apt. 064,...,42.1939,-76.7361,520,Psychotherapist,1991-10-13,798db04aaceb4febd084f1a7c404da93,1371816937,41.747157,-77.584197,0
6,6,2020-06-21 12:15:44,2.131807e+14,fraud_Romaguera Ltd,health_fitness,133.93,Kayla,Sutton,F,010 Weaver Land,...,40.5070,-123.9743,1139,"Therapist, occupational",1951-01-15,17003d7ce534440eadb10c4750e020e5,1371816944,41.499458,-124.888729,0
7,7,2020-06-21 12:15:50,3.589290e+15,fraud_Reichel LLC,personal_care,10.37,Paula,Estrada,F,350 Stacy Glens,...,43.7557,-97.5936,343,"Development worker, international aid",1972-03-05,8be473af4f05fc6146ea55ace73e7ca2,1371816950,44.495498,-97.728453,0
8,8,2020-06-21 12:16:10,3.596357e+15,"fraud_Goyette, Howell and Collier",shopping_pos,4.37,David,Everett,M,4138 David Fall,...,41.0001,-78.2357,3688,Advice worker,1973-05-27,71a1da150d1ce510193d7622e08e784e,1371816970,41.546067,-78.120238,0
9,9,2020-06-21 12:16:11,3.546898e+15,fraud_Kilback Group,food_dining,66.54,Kayla,Obrien,F,7921 Robert Port Suite 343,...,31.6591,-96.8094,263,Barrister,1956-05-30,a7915132c7c4240996ba03a47f81e3bd,1371816971,31.782919,-96.366185,0


In [10]:
data_df.isna().sum()

field1                   0
trans_date_trans_time    0
cc_num                   0
merchant                 0
category                 0
amt                      0
first                    0
last                     0
gender                   0
street                   0
city                     0
state                    0
zip                      0
lat                      0
long                     0
city_pop                 0
job                      0
dob                      0
trans_num                0
unix_time                0
merch_lat                0
merch_long               0
is_fraud                 0
dtype: int64

In [11]:
#convert gender to numeric
gender_dict = {'M': 1,'F': 2}
data_df.gender = [gender_dict[item] for item in data_df.gender]

In [12]:
data_df.head(10)

,field1,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2020-06-21 12:14:25,2.291164e+15,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,1,351 Darlene Green,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,1,2020-06-21 12:14:33,3.573030e+15,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,2,3638 Marsh Union,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2,2020-06-21 12:14:53,3.598215e+15,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,2,9333 Valentine Point,...,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,3,2020-06-21 12:15:15,3.591920e+15,fraud_Haley Group,misc_pos,60.05,Brian,Williams,1,32941 Krystal Mill Apt. 552,...,28.5697,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,4,2020-06-21 12:15:17,3.526826e+15,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,1,5783 Evan Roads Apt. 465,...,44.2529,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0
5,5,2020-06-21 12:15:37,3.040768e+13,fraud_Daugherty LLC,kids_pets,19.55,Danielle,Evans,2,76752 David Lodge Apt. 064,...,42.1939,-76.7361,520,Psychotherapist,1991-10-13,798db04aaceb4febd084f1a7c404da93,1371816937,41.747157,-77.584197,0
6,6,2020-06-21 12:15:44,2.131807e+14,fraud_Romaguera Ltd,health_fitness,133.93,Kayla,Sutton,2,010 Weaver Land,...,40.5070,-123.9743,1139,"Therapist, occupational",1951-01-15,17003d7ce534440eadb10c4750e020e5,1371816944,41.499458,-124.888729,0
7,7,2020-06-21 12:15:50,3.589290e+15,fraud_Reichel LLC,personal_care,10.37,Paula,Estrada,2,350 Stacy Glens,...,43.7557,-97.5936,343,"Development worker, international aid",1972-03-05,8be473af4f05fc6146ea55ace73e7ca2,1371816950,44.495498,-97.728453,0
8,8,2020-06-21 12:16:10,3.596357e+15,"fraud_Goyette, Howell and Collier",shopping_pos,4.37,David,Everett,1,4138 David Fall,...,41.0001,-78.2357,3688,Advice worker,1973-05-27,71a1da150d1ce510193d7622e08e784e,1371816970,41.546067,-78.120238,0
9,9,2020-06-21 12:16:11,3.546898e+15,fraud_Kilback Group,food_dining,66.54,Kayla,Obrien,2,7921 Robert Port Suite 343,...,31.6591,-96.8094,263,Barrister,1956-05-30,a7915132c7c4240996ba03a47f81e3bd,1371816971,31.782919,-96.366185,0


In [13]:
#convert transaction datetime to actual datetime column
data_df.trans_date_trans_time = pd.to_datetime(data_df.trans_date_trans_time)

In [14]:
#convert dob to datetime column
data_df.dob = pd.to_datetime(data_df.dob)

In [15]:
#caclulate age
data_df['age'] = ((data_df.trans_date_trans_time - data_df.dob)/np.timedelta64(1, 'Y')).round()

In [16]:
data_df.head()

,field1,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud,age
0,0,2020-06-21 12:14:25,2.291164e+15,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,1,351 Darlene Green,...,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0,52.0
1,1,2020-06-21 12:14:33,3.573030e+15,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,2,3638 Marsh Union,...,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0,30.0
2,2,2020-06-21 12:14:53,3.598215e+15,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,2,9333 Valentine Point,...,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0,50.0
3,3,2020-06-21 12:15:15,3.591920e+15,fraud_Haley Group,misc_pos,60.05,Brian,Williams,1,32941 Krystal Mill Apt. 552,...,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0,33.0
4,4,2020-06-21 12:15:17,3.526826e+15,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,1,5783 Evan Roads Apt. 465,...,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0,65.0


In [17]:
data_df['tran_day_of_week'] = data_df.trans_date_trans_time.dt.weekday  # weekday
data_df['tran_day_of_month'] = data_df.trans_date_trans_time.dt.day     # day of the month
data_df['tran_month_of_year'] = data_df.trans_date_trans_time.dt.month  # month
data_df['tran_hour_of_day'] = data_df.trans_date_trans_time.dt.hour     # hour

In [18]:
data_df.head()

,field1,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,trans_num,unix_time,merch_lat,merch_long,is_fraud,age,tran_day_of_week,tran_day_of_month,tran_month_of_year,tran_hour_of_day
0,0,2020-06-21 12:14:25,2.291164e+15,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,1,351 Darlene Green,...,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0,52.0,6,21,6,12
1,1,2020-06-21 12:14:33,3.573030e+15,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,2,3638 Marsh Union,...,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0,30.0,6,21,6,12
2,2,2020-06-21 12:14:53,3.598215e+15,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,2,9333 Valentine Point,...,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0,50.0,6,21,6,12
3,3,2020-06-21 12:15:15,3.591920e+15,fraud_Haley Group,misc_pos,60.05,Brian,Williams,1,32941 Krystal Mill Apt. 552,...,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0,33.0,6,21,6,12
4,4,2020-06-21 12:15:17,3.526826e+15,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,1,5783 Evan Roads Apt. 465,...,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0,65.0,6,21,6,12


In [19]:
data_df.category.value_counts()

gas_transport     188029
grocery_pos       176191
home              175460
shopping_pos      166463
kids_pets         161727
shopping_net      139322
entertainment     134118
food_dining       130729
personal_care     130085
health_fitness    122553
misc_pos          114229
misc_net           90654
grocery_net        64878
travel             57956
Name: category, dtype: int64

In [20]:
#convert category to dummies
category_dummies = pd.get_dummies(data_df.category, prefix = 'cat', drop_first = True)
data_df_cat = pd.concat([data_df, category_dummies], axis = 1)

In [21]:
data_df_final = data_df_cat[['amt', 'gender', 'city_pop', 'age', 'tran_day_of_week', 'tran_day_of_month', 'tran_month_of_year',
                            'tran_hour_of_day', 'cat_food_dining', 'cat_gas_transport', 'cat_grocery_net', 'cat_grocery_pos',
                            'cat_health_fitness', 'cat_home', 'cat_kids_pets', 'cat_misc_net', 'cat_misc_pos', 'cat_personal_care',
                            'cat_shopping_net', 'cat_shopping_pos', 'cat_travel', 'is_fraud']]

In [22]:
data_df_final.head()

,amt,gender,city_pop,age,tran_day_of_week,tran_day_of_month,tran_month_of_year,tran_hour_of_day,cat_food_dining,cat_gas_transport,...,cat_health_fitness,cat_home,cat_kids_pets,cat_misc_net,cat_misc_pos,cat_personal_care,cat_shopping_net,cat_shopping_pos,cat_travel,is_fraud
0,2.86,1,333497,52.0,6,21,6,12,0,0,...,0,0,0,0,0,1,0,0,0,0
1,29.84,2,302,30.0,6,21,6,12,0,0,...,0,0,0,0,0,1,0,0,0,0
2,41.28,2,34496,50.0,6,21,6,12,0,0,...,1,0,0,0,0,0,0,0,0,0
3,60.05,1,54767,33.0,6,21,6,12,0,0,...,0,0,0,0,1,0,0,0,0,0
4,3.19,1,1126,65.0,6,21,6,12,0,0,...,0,0,0,0,0,0,0,0,1,0


In [23]:
data_df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1852394 entries, 0 to 1296674
Data columns (total 22 columns):
 #   Column              Dtype  
---  ------              -----  
 0   amt                 float64
 1   gender              int64  
 2   city_pop            int64  
 3   age                 float64
 4   tran_day_of_week    int64  
 5   tran_day_of_month   int64  
 6   tran_month_of_year  int64  
 7   tran_hour_of_day    int64  
 8   cat_food_dining     uint8  
 9   cat_gas_transport   uint8  
 10  cat_grocery_net     uint8  
 11  cat_grocery_pos     uint8  
 12  cat_health_fitness  uint8  
 13  cat_home            uint8  
 14  cat_kids_pets       uint8  
 15  cat_misc_net        uint8  
 16  cat_misc_pos        uint8  
 17  cat_personal_care   uint8  
 18  cat_shopping_net    uint8  
 19  cat_shopping_pos    uint8  
 20  cat_travel          uint8  
 21  is_fraud            int64  
dtypes: float64(2), int64(7), uint8(13)
memory usage: 164.3 MB


In [24]:
data_df_final.to_csv('final_data.csv')